In [88]:
import os
import pandas as pd
import tarfile
import tmkit as tmk
from typing import List, Tuple
# from tmkit.topo import from_pdbtm, from_tmhmm, from_phobius
# from tests import dir_data, tmp_data, exp_data
from tests import tmp_data

# Example dataset

We begin by introducing and downloading the following example dataset to use TMKit. Please see https://tmkit-guide.herokuapp.com/doc/exdataset

In [89]:
tmk.fetch.tmkit_data(
    url='https://sandbox.zenodo.org/record/1219139/files/data.zip?download=1',
    sv_fpn= os.path.join(tmp_data, 'data.zip')
)

===>Dowloading TMKit example dataset...


In [ ]:
# unzip
tmk.fetch.unzip(
    in_fpn= os.path.join(tmp_data, 'data.zip'),
    out_fp= tmp_data
)

# Sequence

## Retrieve

### RCSB PDB file

In [ ]:
new_data_dir = os.path.join(tmp_data, 'new/')
fdir = os.path.join(new_data_dir, 'rcsb/')
os.makedirs(fdir, exist_ok=True)

prot_series = pd.Series(["6e3y", "1xqf"])

tmk.seq.retrieve_pdb_from_rcsb(
    prot_series=prot_series,
    sv_fp= fdir,
)

### PDBTM PDB file

In [ ]:
fdir = os.path.join(new_data_dir, 'pdbtm/')
os.makedirs(fdir, exist_ok=True)

tmk.seq.retrieve_pdb_from_pdbtm(
    prot_series=prot_series,
    sv_fp = fdir,
)

### PDBTM XML file

In [ ]:
fdir = os.path.join(new_data_dir, 'pdbtm/')
os.makedirs(fdir, exist_ok=True)

tmk.seq.retrieve_xml_from_pdbtm(
    prot_series=prot_series,
    sv_fp=fdir
)

### AlphaFold PDB file

#### Download some predicted transmembrane protein structures

In [ ]:
fdir = os.path.join(new_data_dir, 'alphafold/')
os.makedirs(fdir, exist_ok=True)

prot_series = pd.Series(['P63092', 'Q9B6E8', 'P07256', 'P63027'])

tmk.seq.retrieve_pdb_alphafold(
    prot_series=prot_series,
    sv_fp=fdir,
)

#### Using Foldseek for structural alignment of the predicted transmembrane protein structures

In [ ]:
protid = 'P63027'
tmk.seq.retrieve_foldseek(
    pdb_fp= fdir,
    prot_name=protid, # https://alphafold.ebi.ac.uk/entry/P63027
    sv_fp= fdir,
)

# untar the file to a new folder named "P63027_foldseek_result"
fin = os.path.join(fdir, "{}_foldseek_result.gz".format(protid))
with tarfile.open(fin, "r:gz") as tar:
    tar.extractall(path=os.path.join(fdir, "{}_foldseek_result".format(protid)))
    

In [ ]:
# List foldseek result files
!ls -l $fdir/P63027_foldseek_result

## Read

In [ ]:
# Sequence from a Fasta file

fin = os.path.join(tmp_data, "data/fasta/1xqfA.fasta")

sequence = tmk.seq.read_from_fasta(fasta_fpn=fin)

# Get residue IDs from a FASTA file
seq_fasta_ids = tmk.seq.fasid(fasta_fpn=fin)

print(sequence)
print(seq_fasta_ids)


In [ ]:
# Sequence from a PDB file
sequence = tmk.seq.read_from_pdb(
    pdb_fp=os.path.join(tmp_data, "data/pdb/"),
    prot_name='1xqf',
    seq_chain='A',
    file_chain='A',
)

print(sequence)

# Protein topology

## From PDBTM

In [ ]:
topos = {
    'Side1': 'side1',
    'Side2': 'side2',
    'Beta': 'strand',
    'Alpha': 'tmh',
    'Coil': 'coil',
    'Membrane-inside': 'inside',
    'Membrane-loop': 'loop',
    'Interfacial ': 'interfacial',
    'Unknown': 'Unknown',
}


for topo, i in topos.items():
    print('Topology: {}'.format(topo))
    try:
        lower_ids, upper_ids = tmk.topo.from_pdbtm(
            xml_fp=os.path.join(tmp_data, 'data/xml/'),
            prot_name='1xqf',
            seq_chain='A',
            topo=i,
        )
        if lower_ids:
            print('---lower bounds', lower_ids)
            print('---upper bounds', upper_ids)
    except:
        continue
    else:
        print('It does not has this topology.\n')

## From Phobius

In [ ]:
lower_ids, upper_ids = tmk.topo.from_phobius(
    topo='tmh',
    phobius_fpn=os.path.join(tmp_data, 'data/topo/1xqfA.jphobius'),
)
print('---lower bounds', lower_ids)
print('---upper bounds', upper_ids)

## Cytoplasmic or extracellular segments

In [ ]:
pdbtm_seg, pred_seg = tmk.topo.cepdbtm(
  pdb_fp = os.path.join(tmp_data, 'data/pdb/'),
    prot_name='1xqf',
    seq_chain='A',
    file_chain='A',
    topo_fp= os.path.join(tmp_data, 'data/topo/1xqfA.jphobius'),
    xml_fp = os.path.join(tmp_data, 'data/xml/'),
    fasta_fp = os.path.join(tmp_data, 'data/fasta/'),
)
print('---Cytoplasmic and extracellular segments that are structure-derived :\n', pdbtm_seg)
print('---Cytoplasmic and extracellular segments Predicted by the Phobius tool: \n', pred_seg)

# Feature

## Helix surface identification

In [ ]:
# fdir = os.path.join(dir_data, "lips-")

# df = tmk.feature.read_helix_surf(
#     fp=fdir,
#     prot_name='1xqf',
#     file_chain='A',
#     id=1,
# )


# aa_surf_rank, _, _, _ = tmk.feature.read(
#     fp=fdir,
#     prot_name='1xqf',
#     file_chain='A',
# )

# df = tmk.feature.read_helix_all_surf(
#     fp=fdir,
#     prot_name='1xqf',
#     file_chain='A',
# )

# df

In [ ]:
# very long time
# tmk.feature.generate_helix_surfaces(
#   msa_path = os.path.join(tmp_data, 'data/msa/'),
#     prot_name='1xqf',
#     file_chain='A',
#     sv_fp = os.path.join(tmp_data, 'data/lips/'),
# )

In [ ]:
prots = [
    ['1xqf', 'A'],
    ['3pux', 'G'],
    ['3rko', 'A'],
]
df_prot = pd.DataFrame(prots, columns=['prot', 'chain'])
df_prot

In [ ]:
# very long time
# tmk.feature.bgenerate_helix_surfaces(
#   msa_path = os.path.join(tmp_data, 'data/msa/'),
#   sv_fp = os.path.join(tmp_data, 'data/lips/'),
#     df_prot=df_prot,
# )

# CATH

In [ ]:
res = tmk.cath.summary_by_id(
    id='1cukA01'
)


res["domain"] == "http://www.cathdb.info/version/v4_2_0/api/rest/domain_summary/1cukA01"

# MSA

## HHblits

In [ ]:
prots = [
    ['6e3y', 'E'],
    ['6rfq', 'S'],
    ['6t0b', 'm'],
]

df = pd.DataFrame(prots, columns=['prot', 'chain'])
df

In [ ]:
fasta_fp = os.path.join(tmp_data, 'data/fasta/')
hhblits_fp = os.path.join(tmp_data, 'hhblits/bin/')
db_path = os.path.join(tmp_data, 'uniclust_2020.06/UniRef30_2020_06')
sv_fp = os.path.join(tmp_data, 'data/a3m/')

for id in df.index:
    prot_name = df.loc[id, 'prot']
    seq_chain = df.loc[id, 'chain']
    tmk.msa.run_hhblits(
        hhblits_fp=hhblits_fp,
        fasta_fpn=fasta_fp + prot_name + seq_chain + '.fasta',
        sv_fpn=sv_fp + prot_name + seq_chain + '.a3m',
        db_path=db_path,

        # additional parameters
        cpu=2,
        iteration=3,
        maxfilter=100000,
        realign_max=100000,
        all='',
        B=100000,
        Z=100000,
        e=0.001,

        # if you won't do it on clusters, please give False to the parameter send2cloud
        send2cloud=False,
        cloud_cmd="",

        # send2cloud=True,
        # cloud_cmd="qsub -q all.q -N 'jsun'",
    )

## HHfilter

In [ ]:
prots = [
    ['6e3y', 'E'],
    ['6rfq', 'S'],
    ['6t0b', 'm'],
]
import pandas as pd
df = pd.DataFrame(prots, columns=['prot', 'chain'])


hhfilter_fp = './hhblits/bin/'
a3m_path = 'data/a3m/'
new_a3m_path = 'data/a3m/filter/'


hhfilter_fp = os.path.join(tmp_data, 'hhblits/bin/')
a3m_path = os.path.join(tmp_data, 'data/a3m/')
new_a3m_path = os.path.join(tmp_data, 'data/a3m/filter/')

for id in df.index:
    prot_name = df.loc[id, 'prot']
    seq_chain = df.loc[id, 'chain']
    tmk.msa.run_hhfilter(
        hhfilter_fp=hhfilter_fp,
        id=90,
        a3m_fpn=a3m_path + prot_name + seq_chain + '.a3m',
        new_a3m_fpn=new_a3m_path + prot_name + seq_chain + '.a3m',

        # if you won't do it on clusters, please give False to the parameter send2cloud
        send2cloud=False,
        cloud_cmd="",

        # send2cloud=True,
        # cloud_cmd="qsub -q all.q -N 'jsun'",
    )

# Collation

In [ ]:
# PDBTM

pdb_rcsb_fp = os.path.join(tmp_data, 'data/pdb/collate/rcsb/')
pdb_pdbtm_fp = os.path.join(tmp_data, 'data/pdb/collate/pdbtm/')

chains = tmk.collate.chain(
    prot_name='6cxh',
    pdb_fp=pdb_pdbtm_fp,
)
print(chains)

# Edge

## bipartite

In [ ]:
from tmkit.sequence import Fasta as sfasta
from tmkit.seqnetrr.combo.Length import length as pl
from tmkit.seqnetrr.combo.Position import Position as pfasta
from tmkit.seqnetrr.window.Pair import Pair
from tmkit.seqnetrr.graph.Bipartite import Bipartite as bigraph

# read a sequence
sequence = sfasta.get(
  fasta_fpn = os.path.join(tmp_data, 'data/fasta/1xqfA.fasta')
)
sequence


# generate residue pairs according to sequence separation
pos_list = pl(
    seq_sep_superior=None,
    seq_sep_inferior=0
).to_pair(
    length=len(sequence)
)
pos_list[:10]



position = pfasta(
    sequence=sequence,
).pair(
    pos_list=pos_list,
)
position[:10]



window_m_ids = Pair(
    sequence=sequence,
    position=position,
    window_size=5,
).mid()
window_m_ids[:10]



res = bigraph(
    sequence=sequence,
    window_size=5,
    window_m_ids=window_m_ids,
    kind='patch',
    patch_size=2,
    input_kind='simulate',
).assign(
    list_2d=position,
    simu_seq_len=len(sequence),
    mode='hash',
)
# print(res)



res = bigraph(
    sequence=sequence,
    window_size=5,
    window_m_ids=window_m_ids,
    kind='patch',
    patch_size=2,
    input_kind='simulate',
).assign(
    list_2d=position,
    simu_seq_len=len(sequence),
    mode='hash',
)
# res

## Unigraph - Pipeline

In [ ]:
from tmkit.sequence import Fasta as sfasta

# read a sequence
sequence = sfasta.get(
  fasta_fpn = os.path.join(tmp_data, 'data/fasta/1xqfA.fasta')
)
print(sequence)

In [ ]:
from tmkit.seqnetrr.combo.Length import length as pl

# generate residue pairs according to sequence separation
pos_list = pl(
    seq_sep_superior=None,
    seq_sep_inferior=0
).to_pair(
    length=len(sequence)
)
pos_list[:10]

In [ ]:
from tmkit.seqnetrr.combo.Position import Position as pfasta

position = pfasta(
    sequence=sequence,
).pair(
    pos_list=pos_list,
)
print(position[:10])

In [ ]:
from tmkit.seqnetrr.window.Pair import Pair

window_m_ids = Pair(
    sequence=sequence,
    position=position,
    window_size=5,
).mid()
print(window_m_ids[:10])

In [ ]:
from tmkit.seqnetrr.graph.Unipartite import Unipartite as unigraph

res = unigraph(
    sequence=sequence,
    window_size=5,
    window_m_ids=window_m_ids,
    input_kind='freecontact',
).assign(
    list_2d=position,
    fpn= os.path.join(tmp_data, 'data/rrc/tool/1xqfA.evfold'),
    mode='hash',
)

In [ ]:
from tmkit.seqnetrr.graph.Unipartite import Unipartite as unigraph

res = unigraph(
    sequence=sequence,
    window_size=5,
    window_m_ids=window_m_ids,
    input_kind='simulate',
).assign(
    list_2d=position,
    simu_seq_len=len(sequence),
    mode='hash',
)

## Unigraph - Pipeline2

In [ ]:
from tmkit.sequence import Fasta as sfasta
# read a sequence
sequence = sfasta.get(
  fasta_fpn = os.path.join(tmp_data, 'data/fasta/1xqfA.fasta')
)
print(sequence)

In [ ]:
from tmkit.seqnetrr.combo.Length import length as plength

pos_list = plength(
    seq_sep_inferior=0,
    seq_sep_superior=None,
).tosgl(
    length=len(sequence),
)
print(pos_list[:10])

In [ ]:
from tmkit.seqnetrr.combo.Position import Position as pfasta

position = pfasta(
    sequence=sequence,
).single(
    pos_list=pos_list,
)
print(position[:10])

In [ ]:
from tmkit.seqnetrr.window.Single import Single

window_m_ids = Single(
    sequence=sequence,
    position=position,
    window_size=3,
).mid()
print(window_m_ids[:10])

In [ ]:
from tmkit.seqnetrr.graph.Cumulative import Cumulative

res = Cumulative(
    sequence=sequence,
    window_size=5,
    window_m_ids=window_m_ids,
    input_kind='freecontact',
).assign(
    list_2d=position,
    L=int(len(sequence)/5),
    fpn = os.path.join(tmp_data, 'data/rrc/tool/1xqfA.evfold')
)

In [ ]:
from tmkit.seqnetrr.graph.Cumulative import Cumulative

res = Cumulative(
    sequence=sequence,
    window_size=5,
    window_m_ids=window_m_ids,
    input_kind='simulate',
).assign(
    list_2d=position,
    L=int(len(sequence)/5),
    simu_seq_len=len(sequence),
)
print(res[:10])

# Mapping

In [ ]:

res = tmk.mapping.pdb2uniprot(
    id='101m.A',
    ref_fpn= os.path.join(tmp_data, 'data/map/pdb_chain_uniprot.csv'),
)
print(res)

# Mutation

In [ ]:
# tmk.mut.download_predmuthtp_db(
#     sv_fp= os.path.join(new_data_dir, 'ppi/mutation')
# )

# PPI

In [ ]:
# tmk.ppi.download_intact_db(
#     version='current',
#     sv_fp= os.path.join(new_data_dir, 'ppi')
# )

# Residue contact

In [ ]:
import tmkit as tmk

df1 = tmk.rrc.read(
    prot_name='1xqf',
    seq_chain='A',
    fasta_fp = os.path.join(tmp_data, 'data/fasta/'),
    pdb_fp = os.path.join(tmp_data, 'data/pdb/'),
    xml_fp = os.path.join(tmp_data, 'data/xml/'),
    dist_fp = os.path.join(tmp_data, 'data/rrc/'),
    tool_fp = os.path.join(tmp_data, 'data/rrc/tool/'),
    seq_sep_inferior=1,
    seq_sep_superior=None,
    tool='membrain2',
)
df1